### SparkContext Configurations ###

In [0]:
# https://spark.apache.org/docs/latest/configuration.html#viewing-spark-properties

### SparkSession ###

Create a DataFrame/Dataset from a collection (e.g. list or set)

In [0]:
from pyspark.sql import SparkSession

# create dataframe from list of dicts
spark = SparkSession.builder.appName("My Spark App").getOrCreate()
data = [{'name':'Alice','age':4}, {'name':'Bob','age':8}]
df = spark.createDataFrame(data, ['name','age'])
df.collect()

/databricks/spark/python/pyspark/sql/session.py:355: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
 warnings.warn("inferring schema from dict is deprecated,"
Out[1]: [Row(name=4, age='Alice'), Row(name=8, age='Bob')]

In [0]:
# create dataframe from list of tuples
data = [('Alice',4), ('Bob',8)]
df = spark.createDataFrame(data, ['name','age'])
df.collect()

Out[3]: [Row(name='Alice', age=4), Row(name='Bob', age=8)]

In [0]:
# create dataframe from RDD and schema as Row object
from pyspark.sql import Row
data = [('Alice',4), ('Bob',8)]
rdd = sc.parallelize(data)
Person = Row('name', 'age')
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.collect()

Out[4]: [Row(name='Alice', age=4), Row(name='Bob', age=8)]

Create a DataFrame for a range of numbers

In [0]:
df = spark.range(start=5, end=21, step=5, numPartitions=None)

Access the DataFrameReaders

In [0]:
spark.read.csv # also json, parquet, orc, jdbc

Out[48]: <bound method DataFrameReader.csv of <pyspark.sql.readwriter.DataFrameReader object at 0x7f061752d390>>

Register User Defined Functions (UDFs)

In [0]:
def sq(x):
  return x*x

spark.udf.register(name="sq", f=my_squared, returnType=IntegerType())
spark.range(5, 21, 5).registerTempTable("test")
spark.sql("SELECT id, sq(id) FROM test").show()

+---+------+
 id|sq(id)|
+---+------+
 5| 25|
 10| 100|
 15| 225|
 20| 400|
+---+------+

### DataFrameReader ###

Read data for the “core” data formats (CSV, JSON, JDBC, ORC, Parquet, text and tables)

In [0]:
# read Json 3 ways
df = spark.read.json("/FileStore/tables/people.json")
df = spark.read.load(path="/FileStore/tables/people.json", format="json")
df = spark.read.format("json").load(path="/FileStore/tables/people.json")
df.show()

+---+------+
age| name|
+---+------+
 23|Angela|
 10| Bob|
 44| Cathy|
 39| Dylan|
 56| Elena|
+---+------+

In [0]:
# read CSV
df3 = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/people.csv")

In [0]:
# read using JDBC
remote_table = spark.read.format("jdbc")\
  .option("driver", "org.postgresql.Driver")\
  .option("url", "jdbc:postgresql://database_server")\
  .option("dbtable", "schema.tablename")\
  .option("user", "me")\
  .option("password", "12345")\
  .load()

# read ORC
df5 = spark.read.orc("/path/to/file.orc")
# read Parquet
df6 = spark.read.parquet("/path/to/file.parquet")

How to configure options for specific formats

In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/people.csv")

How to read data from non-core formats using format() and load()

In [0]:
# format()
df = spark.read.format('json').load('/FileStore/tables/people.json')

In [0]:
# load()
df = spark.read.load(path='/FileStore/tables/people.json', format='json')
df.show()

+---+------+
age| name|
+---+------+
 23|Angela|
 10| Bob|
 44| Cathy|
 39| Dylan|
 56| Elena|
+---+------+

How to specify a DDL-formatted schema

In [0]:
df = spark.read.csv(path='/FileStore/tables/people.csv', header=True, schema='NAME VARCHAR(255), AGE INTEGER')
df.show()

+------+---+
 NAME|AGE|
+------+---+
Angela| 23|
 Bob| 10|
 Cathy| 44|
 Dylan| 39|
 Elena| 56|
+------+---+

How to construct and specify a schema using the StructType classes

In [0]:
from pyspark.sql.types import *
mySchema = StructType([StructField("name", StringType(), True), StructField("age", IntegerType(), True)])
df = spark.read.load(path='/FileStore/tables/people.json', format='json', schema=mySchema)
df.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3982.load.
: org.apache.spark.sql.AnalysisException: Path does not exist: dbfs:/FileStore/tables/people.json;
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:614)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:597)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.immutable.List.flatMap(List.scala:355)
	at org.apache.spark.sql.execution.datasources.DataSource.org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary(DataSource.scala:597)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:391)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:307)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:293)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:214)
	at sun.reflect.GeneratedMethodAccessor477.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-2248903751822491> in <module> 
 1 from pyspark . sql . types import * 
 2 mySchema = StructType ( [ StructField ( "name" , StringType ( ) , True ) , StructField ( "age" , IntegerType ( ) , True ) ] ) 
 ----> 3 df = spark . read . load ( path = '/FileStore/tables/people.json' , format = 'json' , schema = mySchema ) 
 4 df . show ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 164 self . options ( ** options ) 
 165 if isinstance ( path , basestring ) : 
 --> 166 return self . _df ( self . _jreader . load ( path ) ) 
 167 elif path is not None : 
 168 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': 

### DataFrameWriter ###

Write data to the “core” data formats (csv, json, jdbc, orc, parquet, text and tables)

In [0]:
help(dbutils.fs.rm)


Help on method f_with_exception_handling in module dbutils:

f_with_exception_handling(*args, **kwargs) method of dbutils.FSHandler instance

In [0]:
df.write.parquet('/FileStore/tables/output/people.parquet', mode='Append')
df_written = spark.read.parquet('/FileStore/tables/output/people.parquet')
df_written.show()

+---+------+
age| name|
+---+------+
 23|Angela|
 10| Bob|
 44| Cathy|
 39| Dylan|
 56| Elena|
 23|Angela|
 10| Bob|
 44| Cathy|
 39| Dylan|
 56| Elena|
+---+------+

Overwriting existing files

In [0]:
df.write.parquet('/FileStore/tables/output/people.parquet', mode='Overwrite')

How to configure options for specific formats

In [0]:
df.write.options(header=True, sep='|').csv('/FileStore/tables/output/people.csv', mode='Overwrite')
df_written = spark.read.option('header', True).option('sep', '|').csv('/FileStore/tables/output/people.csv')
df_written.show()

+---+------+
age| name|
+---+------+
 23|Angela|
 10| Bob|
 44| Cathy|
 39| Dylan|
 56| Elena|
+---+------+

How to write a data source to 1 single file or N separate files

In [0]:
df.coalesce(1).write.csv('/FileStore/tables/output/csv_1_partition/', mode='Overwrite')
df.repartition(5).write.csv('/FileStore/tables/output/csv_5_partitions/', mode='Overwrite')

In [0]:
%fs ls /FileStore/tables/output/csv_1_partition/

path,name,size
dbfs:/FileStore/tables/output/csv_1_partition/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/tables/output/csv_1_partition/_committed_6041240411489043784,_committed_6041240411489043784,113
dbfs:/FileStore/tables/output/csv_1_partition/_committed_8692727945481414173,_committed_8692727945481414173,212
dbfs:/FileStore/tables/output/csv_1_partition/_started_6041240411489043784,_started_6041240411489043784,0
dbfs:/FileStore/tables/output/csv_1_partition/_started_8692727945481414173,_started_8692727945481414173,0
dbfs:/FileStore/tables/output/csv_1_partition/part-00000-tid-8692727945481414173-5bdeee7b-8825-4ff0-8c83-a22f45b6e657-321-1-c000.csv,part-00000-tid-8692727945481414173-5bdeee7b-8825-4ff0-8c83-a22f45b6e657-321-1-c000.csv,44


In [0]:
%fs ls /FileStore/tables/output/csv_5_partitions/

path,name,size
dbfs:/FileStore/tables/output/csv_5_partitions/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/tables/output/csv_5_partitions/_committed_1336251933358378223,_committed_1336251933358378223,924
dbfs:/FileStore/tables/output/csv_5_partitions/_committed_4894914513389392256,_committed_4894914513389392256,469
dbfs:/FileStore/tables/output/csv_5_partitions/_started_1336251933358378223,_started_1336251933358378223,0
dbfs:/FileStore/tables/output/csv_5_partitions/_started_4894914513389392256,_started_4894914513389392256,0
dbfs:/FileStore/tables/output/csv_5_partitions/part-00000-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-323-1-c000.csv,part-00000-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-323-1-c000.csv,9
dbfs:/FileStore/tables/output/csv_5_partitions/part-00001-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-324-1-c000.csv,part-00001-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-324-1-c000.csv,7
dbfs:/FileStore/tables/output/csv_5_partitions/part-00002-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-325-1-c000.csv,part-00002-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-325-1-c000.csv,9
dbfs:/FileStore/tables/output/csv_5_partitions/part-00003-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-326-1-c000.csv,part-00003-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-326-1-c000.csv,9
dbfs:/FileStore/tables/output/csv_5_partitions/part-00004-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-327-1-c000.csv,part-00004-tid-1336251933358378223-33fcc9c8-146d-4d9a-bbde-e938b769334f-327-1-c000.csv,10


In [0]:
df_out = spark.read.format('csv').load('/FileStore/tables/output/partitioned_csv/part-00001-tid-321870977250130119-fc361c0e-aad6-4a4b-ae42-d341000a1a02-271-1-c000.csv')
df_out.show()

+---+---+
_c0|_c1|
+---+---+
 10|Bob|
+---+---+

How to write partitioned data

In [0]:
df.repartition(5).write.parquet('/FileStore/tables/output/partitioned/', mode='Overwrite')
display(dbutils.fs.ls('/FileStore/tables/output/partitioned/'))

How to bucket data by a given set of columns

In [0]:
# save as bucketed table file
# WARNING - cannot just do .bucketBy(...).save(path='/path/to/out/file') - MUST specify path in .option()
df.write.format('parquet')\
  .bucketBy(numBuckets=5, col='name')\
  .mode('overwrite')\
  .option('path', '/FileStore/tables/output/bucketed/')\
  .saveAsTable('sorted_bucketed_table')

In [0]:
%fs ls /FileStore/tables/output/bucketed/

In [0]:
%sql SELECT * from sorted_bucketed_table

age,name
56,Elena
39,Dylan
10,Bob
44,Cathy
23,Angela


In [0]:
# can save as just a table
df.write.format('parquet')\
  .bucketBy(numBuckets=5, col='name')\
  .mode('overwrite')\
  .saveAsTable('sorted_bucketed_table')

### DataFrame ###

Have a working understanding of every action such as take(), collect(), and foreach()

In [0]:
display(df.take(3))

age,name
23,Angela
10,Bob
44,Cathy


In [0]:
# foreach() - can be used to force-update an accumulator
total_age = sc.accumulator(0)

def print_row(row):
  total_age.add(row.age)
  
df.foreach(print_row) # gets printed to stdout of each executor, not driver
print(total_age.value)

172

Produce a distinct set

In [0]:
df = sc.parallelize([2,2,4,1,1,4,6])
df.distinct().collect()

Out[5]: [1, 2, 4, 6]

Filtering data

In [0]:
df = spark.createDataFrame([('a',1), ('b',2), ('c',5), ('d',9), ('e',10)], ('letter', 'number'))
display(df.filter(df.number > 7).collect())

letter,number
d,9
e,10


Change number of partitions using repartition() and coalesce()

In [0]:
df2 = df.repartition(5)
df2.rdd.getNumPartitions()

Out[15]: 5

In [0]:
df3 = df2.coalesce(2)
df3.rdd.getNumPartitions()

Out[17]: 2

Joins

In [0]:
employees = spark.createDataFrame([('Ana', 'E1'), ('Bob', 'E2'), ('Charlie', 'E3')], ('Employee', 'EID'))
orders = spark.createDataFrame([('Order1', 'E1'), ('Order2', 'E2'), ('Order3', 'E3'), ('Order4', 'E1'), ('Order5', 'E2'), ('Order6', 'E2')], ('Order', 'EID'))
display(employees.join(orders, employees['EID'] == orders['EID']).select(collect())

Employee,EID,Order,EID
Charlie,E3,Order3,E3
Ana,E1,Order1,E1
Ana,E1,Order4,E1
Bob,E2,Order2,E2
Bob,E2,Order5,E2
Bob,E2,Order6,E2


Unions

In [0]:
employees = spark.createDataFrame([('Ana', 'E1'), ('Bob', 'E2'), ('Charlie', 'E3')], ('Employee', 'EID'))
employees2 = spark.createDataFrame([('Dilbert', 'E8'), ('Egg', 'E11'), ('French', 'E8')], ('Employee', 'EID'))
display(employees.union(employees2).collect())

Employee,EID
Ana,E1
Bob,E2
Charlie,E3
Dilbert,E8
Egg,E11
French,E8


Aggregates

In [0]:
df = spark.createDataFrame([('a',1), ('b',2), ('c',5), ('d',9), ('e',10)], ('letter', 'number'))
df.agg({'number':'max'}).collect()

Out[2]: [Row(max(number)=10)]

In [0]:
from pyspark.sql import functions
df.agg(functions.max('number')).collect()

Out[31]: [Row(max(number)=10)]

Converting a DataFrame to a global or temp view.

In [0]:
df.createTempView("myTempView")
df.createGlobalTempView("myGlobalTempView")

Applying broadcast hints

In [0]:
from pyspark.sql.functions import broadcast
employees = spark.createDataFrame([('Ana', 'E1'), ('Bob', 'E2'), ('Charlie', 'E3')], ('Employee', 'EID'))
orders = spark.createDataFrame([('Order1', 'E1'), ('Order2', 'E2'), ('Order3', 'E3'), ('Order4', 'E1'), ('Order5', 'E2'), ('Order6', 'E2')], ('Order', 'EID'))
broadcast(employees).join(orders, employees.EID == orders.EID).show()

+--------+---+------+---+
Employee|EID| Order|EID|
+--------+---+------+---+
 Ana| E1|Order1| E1|
 Bob| E2|Order2| E2|
 Charlie| E3|Order3| E3|
 Ana| E1|Order4| E1|
 Bob| E2|Order5| E2|
 Bob| E2|Order6| E2|
+--------+---+------+---+

In [0]:
# another way, using table views
employees.createOrReplaceTempView("employees")
orders.createOrReplaceTempView("orders")
broadcast(spark.table("employees")).join(spark.table("orders"), "EID").show()

+---+--------+------+
EID|Employee| Order|
+---+--------+------+
 E1| Ana|Order1|
 E2| Bob|Order2|
 E3| Charlie|Order3|
 E1| Ana|Order4|
 E2| Bob|Order5|
 E2| Bob|Order6|
+---+--------+------+

### Row & Column

Create a dataframe from Json

In [0]:
# create dataframe from json
from pyspark.sql.types import StructField, StringType, IntegerType, StructType 
schema = [StructField('name', StringType(), True), StructField('age', IntegerType(), True)]
schema_struct = StructType(fields=schema)
df = spark.read.json("/FileStore/tables/people.json", schema=schema_struct)
display(df.collect())

name,age
Angela,23
Bob,10
Cathy,44
Dylan,39
Elena,56


Compute and add a new column

In [0]:
df_new = df.withColumn('doubleage',df['age']*2)
df_new.show()

+------+---+---------+
 name|age|doubleage|
+------+---+---------+
Angela| 23| 46|
 Bob| 10| 20|
 Cathy| 44| 88|
 Dylan| 39| 78|
 Elena| 56| 112|
+------+---+---------+

Rename a column

In [0]:
df_new.withColumnRenamed('doubleage', 'double_age').show()

+------+---+----------+
 name|age|double_age|
+------+---+----------+
Angela| 23| 46|
 Bob| 10| 20|
 Cathy| 44| 88|
 Dylan| 39| 78|
 Elena| 56| 112|
+------+---+----------+

Drop a column

In [0]:
df_new.drop('double_age').show()

+------+---+---------+
 name|age|doubleage|
+------+---+---------+
Angela| 23| 46|
 Bob| 10| 20|
 Cathy| 44| 88|
 Dylan| 39| 78|
 Elena| 56| 112|
+------+---+---------+

Select row based on condition

In [0]:
result = df.filter(df.age > 40).collect()

Turn row into dictionary

In [0]:
row = result[0]
row

Out[37]: Row(name='Cathy', age=44)

In [0]:
row.asDict('name')

Out[38]: {'name': 'Cathy', 'age': 44}

In [0]:
row.asDict().keys()

Out[40]: dict_keys(['name', 'age'])

In [0]:
row.asDict().values()

Out[41]: dict_values(['Cathy', 44])

### Spark SQL Functions

Aggregate functions: getting the first or last item from an array

In [0]:
# example 1: select from array
from pyspark.sql.functions import element_at
df = spark.createDataFrame([(["a", "b", "c"],), ([],)], ['col1'])
df.show()

+---------+
 col1|
+---------+
[a, b, c]|
 []|
+---------+

In [0]:
# get first element
df.select(element_at(df.col1, 1)).show()

+-------------------+
element_at(col1, 1)|
+-------------------+
 a|
 null|
+-------------------+

In [0]:
# get last element
df.select(element_at(df.col1, -1)).show()

+--------------------+
element_at(col1, -1)|
+--------------------+
 c|
 null|
+--------------------+

In [0]:
# example 2: select from map
df = spark.createDataFrame([({"a":1, "b":2},), ({"b":3, "d":4},)], ["col1"])
df.show()

+----------------+
 col1|
+----------------+
[a -> 1, b -> 2]|
[d -> 4, b -> 3]|
+----------------+

In [0]:
# lookup element using key
result = df.select(element_at(df.col1, "b").alias("b_value"))
result.show()

+-------+
b_value|
+-------+
 2|
 3|
+-------+

In [0]:
# example 3: split an array element
from pyspark.sql import Row
row = [Row(col1="a", col2="b", col3=["c", "d"])]
rdd = sc.parallelize(row)
df = spark.createDataFrame(rdd)
df.show()

+----+----+------+
col1|col2| col3|
+----+----+------+
 a| b|[c, d]|
+----+----+------+

In [0]:
# split column 3 into two separate columns
result = df.withColumn("col4", df["col3"].getItem(1))\
           .withColumn("col3", df["col3"].getItem(0))
result.show()

+----+----+----+----+
col1|col2|col3|col4|
+----+----+----+----+
 a| b| c| d|
+----+----+----+----+

Aggregate functions: computing the min and max values of a column

In [0]:
from pyspark.sql import functions as F
df = spark.createDataFrame([(10,), (2,), (7,)], ["nums"])
df.select(F.min(df.nums), F.max(df.nums)).show()

+---------+---------+
min(nums)|max(nums)|
+---------+---------+
 2| 10|
+---------+---------+

Collection functions: testing if an array contains a value

In [0]:
df.select(df.nums, array_contains(df.nums, 2).alias("has_2")).show()

+---------+-----+
 nums|has_2|
+---------+-----+
[1, 2, 3]| true|
[2, 3, 6]| true|
 [4, 6]|false|
+---------+-----+

Collection functions: exploding or flattening data

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import explode
df = spark.createDataFrame([Row(mylist=[1,2,3]), Row(mylist=[4,5,6])])
df.select(explode(df.mylist)).collect()

Out[21]: [Row(col=1), Row(col=2), Row(col=3), Row(col=4), Row(col=5), Row(col=6)]

In [0]:
df = spark.createDataFrame([([[1, 2, 3], [4, 5], [6]],), ([None, [4, 5]],)], ['data'])
display(df.collect())

data
"List(List(1, 2, 3), List(4, 5), List(6))"
"List(null, List(4, 5))"


In [0]:
from pyspark.sql.functions import flatten
df.select(flatten(df.data)).show()  # null element in array results in null flattened result

+------------------+
 flatten(data)|
+------------------+
[1, 2, 3, 4, 5, 6]|
 null|
+------------------+

In [0]:
# instead of null element in array, use empty array
df = spark.createDataFrame([([[1, 2, 3], [4, 5], [6]],), ([[None], [4, 5]],)], ['data'])
df.select(flatten(df.data)).show() 

+------------------+
 flatten(data)|
+------------------+
[1, 2, 3, 4, 5, 6]|
 [, 4, 5]|
+------------------+

Date time functions: parsing strings into timestamps or formatting timestamps into strings

In [0]:
from pyspark.sql.functions import from_unixtime

# convert from epoch milliseconds to string representation
spark.conf.set("spark.sql.session.timeZone", "America/Los_Angeles")
time_df = spark.createDataFrame([(1428476400,)], ['unix_time'])
time_df.select(from_unixtime('unix_time').alias('ts')).collect()

Out[35]: [Row(ts='2015-04-08 00:00:00')]

In [0]:
# convert from string representation to time object in specified timezone
from pyspark.sql.functions import from_utc_timestamp
df = spark.createDataFrame([('1997-02-28 10:30:00', 'JST')], ['timestamp', 'timezone'])
df.select(from_utc_timestamp(df.timestamp, df.timezone).alias('local_time')).collect()

Out[42]: [Row(local_time=datetime.datetime(1997, 2, 28, 19, 30))]

In [0]:
# convert from string representation of local time to time object in utc time
from pyspark.sql.functions import to_utc_timestamp
df = spark.createDataFrame([('1997-02-28 10:30:00', 'JST')], ['ts', 'tz'])
df.select(to_utc_timestamp(df.ts, df.tz).alias('utc_time')).collect()

Out[46]: [Row(utc_time=datetime.datetime(1997, 2, 28, 1, 30))]

In [0]:
# convert from string representation to time object by specifying format of string
from pyspark.sql.functions import to_date
df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['timestamp'])
df.select(to_date(df.timestamp, 'yyyy-MM-dd HH:mm:ss').alias('date')).collect()

Out[44]: [Row(date=datetime.date(1997, 2, 28))]

Math functions: computing the cosine, floor or log of a number

In [0]:
from pyspark.sql.functions import cos, floor, log
df = spark.createDataFrame([(3,), (6,)], ["data"])
df.select(cos(df.data).alias("cosine"), floor(df.data).alias("floor"), log(10.0, df.data).alias("log10")).show()

+-------------------+-----+-------------------+
 cosine|floor| log10|
+-------------------+-----+-------------------+
-0.9899924966004454| 3|0.47712125471966244|
 0.9601702866503661| 6| 0.7781512503836435|
+-------------------+-----+-------------------+

Misc functions: converting a value to crc32, md5, sha1 or sha2

In [0]:
from pyspark.sql.functions import crc32, md5, sha1, sha2

spark.createDataFrame([('abcdef',)], ['data']).select(crc32('data'), md5('data'), sha1('data'), sha2('data', numBits=256)).show()

+-----------+--------------------+--------------------+--------------------+
crc32(data)| md5(data)| sha1(data)| sha2(data, 256)|
+-----------+--------------------+--------------------+--------------------+
 1267612143|e80b5017098950fc5...|1f8ac10f23c5b5bc1...|bef57ec7f53a6d40b...|
+-----------+--------------------+--------------------+--------------------+

Test if a column is null or not-null, NAN

In [0]:
from pyspark.sql import Row
df = spark.createDataFrame([Row(name='Alice', age=10), Row(name='Bob', age=11), Row(name='Charlie', age=None)])
df.filter(df.age.isNull()).show()

+----+-------+
 age| name|
+----+-------+
null|Charlie|
+----+-------+

In [0]:
from pyspark.sql.functions import isnull
df.select(isnull(df.age)).show()

+-------------+
(age IS NULL)|
+-------------+
 false|
 false|
 true|
+-------------+

In [0]:
df.filter(df.age.isNotNull()).show()

+---+-----+
age| name|
+---+-----+
 10|Alice|
 11| Bob|
+---+-----+

Sorting functions: sorting data in descending order, ascending order, and sorting with proper null handling

In [0]:
df = spark.createDataFrame([Row(name='Alice', age=10), Row(name='Bob', age=11), Row(name='Charlie', age=None)])
df.sort(df.age.asc()).show()

+----+-------+
 age| name|
+----+-------+
null|Charlie|
 10| Alice|
 11| Bob|
+----+-------+

In [0]:
df.sort(df.age.desc_nulls_last()).show()

+----+-------+
 age| name|
+----+-------+
 11| Bob|
 10| Alice|
null|Charlie|
+----+-------+

UDF functions: employing a UDF function.

In [0]:
from pyspark.sql.types import IntegerType

str_len_udf = udf(lambda str: len(str), IntegerType())
df = spark.createDataFrame([Row(name='Alice', age=10), Row(name='Bob', age=11), Row(name='Charlie', age=None)])
df.select(df.name, str_len_udf(df.name).alias('name_length')).show()

+-------+-----------+
 name|name_length|
+-------+-----------+
 Alice| 5|
 Bob| 3|
Charlie| 7|
+-------+-----------+

In [0]:
# same thing with SQL
df.createOrReplaceTempView("people")
spark.udf.register("str_len_udf", lambda str: len(str))
spark.sql("SELECT name, str_len_udf(name) FROM people").show()

+-------+-----------------+
 name|str_len_udf(name)|
+-------+-----------------+
 Alice| 5|
 Bob| 3|
Charlie| 7|
+-------+-----------------+

String functions: applying a provided regular expression

In [0]:
df = spark.createDataFrame([("Alice", 1), ("Bob", 2), ("Charlie",  3)], ["name", "age"])
regex = 'ice$'
df.filter(df.name.rlike(regex)).collect()

Out[7]: [Row(name='Alice', age=1)]

In [0]:
# select a column based on regex for column name
df = spark.createDataFrame([("a", 1), ("b", 2), ("c",  3)], ["Col1", "Col2"])
df.select(df.colRegex("`(Col1)?+.+`")).show()

+----+
Col2|
+----+
 1|
 2|
 3|
+----+

String functions: trimming strings

In [0]:
from pyspark.sql.functions import rtrim
df = spark.createDataFrame([("       Alice", 1), ("Bob        ", 2), ("    Charlie",  3)], ["name", "age"])
df.select(rtrim(df.name)).collect()

Out[19]: [Row(rtrim(name)=' Alice'),
 Row(rtrim(name)='Bob'),
 Row(rtrim(name)=' Charlie')]

In [0]:
from pyspark.sql.functions import ltrim
df = spark.createDataFrame([("       Alice", 1), ("Bob        ", 2), ("    Charlie",  3)], ["name", "age"])
df.select(ltrim(df.name)).collect()

Out[18]: [Row(ltrim(name)='Alice'),
 Row(ltrim(name)='Bob '),
 Row(ltrim(name)='Charlie')]

String functions: extracting substrings

In [0]:
df = spark.createDataFrame([("Alice", 1), ("Bob", 2), ("Charlie",  3)], ["name", "age"])
df.select(df.name.substr(0,3)).collect()

Out[9]: [Row(substring(name, 0, 3)='Ali'),
 Row(substring(name, 0, 3)='Bob'),
 Row(substring(name, 0, 3)='Cha')]